<a href="https://colab.research.google.com/github/Thanh-Hoo/CS114.K21/blob/master/Sarcasm_Headlines_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount( '/content/gdrive' )

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import pandas as pd

In [3]:
!unzip 'gdrive/My Drive/DATA_COLAB/BC_ML/news-headlines-dataset-for-sarcasm-detection.zip'

Archive:  gdrive/My Drive/DATA_COLAB/BC_ML/news-headlines-dataset-for-sarcasm-detection.zip
  inflating: Sarcasm_Headlines_Dataset.json  
  inflating: Sarcasm_Headlines_Dataset_v2.json  


In [28]:
df_1 = pd.read_json("Sarcasm_Headlines_Dataset.json",lines=True)

In [85]:
df_2 = pd.read_csv("gdrive/My Drive/DATA_COLAB/BC_ML/test_data.csv",)
df_2

,headline,is_sarcastic
0,BirthdayForBreonna Marks What Would've Been Br...,0
1,TheShowMustBePaused Was Eclipsed By #BlackOutT...,0
2,"Again, Again, Again!' Exclaims Clapping, Grinn...",1
3,"And Then There Were 23,' Says Wayne Messam Cro...",1
4,And Then Those 12 People Send It To 12 People ...,1
...,...,...
2154,Zoom App Sued For Sharing User Data,1
2155,Zoom Can Track Who's Not Paying Attention In Y...,0
2156,Zoom Crasher Becomes Too Engrossed In Sales Me...,1
2157,Zuckerberg Says He'll Review Policies That All...,0


In [32]:
df_1.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [33]:
df_1.info()
print("                  ")
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26709 entries, 0 to 26708
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   article_link  26709 non-null  object
 1   headline      26709 non-null  object
 2   is_sarcastic  26709 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 626.1+ KB
                  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2159 entries, 0 to 2158
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   headline      2159 non-null   object
 1   is_sarcastic  2159 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 33.9+ KB


In [114]:
df_1=df_1.drop(labels="article_link",axis=1)

In [36]:
import numpy as np
from sklearn.model_selection import train_test_split
#from sklearn.metrics import accuracy_score
import string 
from string import digits
from string import punctuation
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
for i in df_1['headline']:
  i = str(i)

for a in df_2['headline']:
  a = str(a)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [115]:
df_1

,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0
...,...,...
26704,american politics in moral free-fall,0
26705,america's best 20 hikes,0
26706,reparations and obama,0
26707,israeli ban targeting boycott supporters raise...,0


In [116]:
#clean the data remove all digits and punctuation
hl_clean_1 = []
hl_clean_2 = []

for i in df_1['headline']:
    clean = i.translate(str.maketrans('', '', punctuation))
    clean = clean.translate(str.maketrans('', '', digits))
    hl_clean_1.append(clean)

for a in df_2['headline']:
    clean = a.translate(str.maketrans('', '', punctuation))
    clean = clean.translate(str.maketrans('', '', digits))
    hl_clean_2.append(clean)


In [117]:
#tonization, break the headline to the lonely character like noun, pharse, ....
hl_tokens_1 = []
hl_tokens_2 = []
for i in hl_clean_1:
    i = i.split()
    hl_tokens_1.append(i)

for i in hl_clean_2:
    i = i.split()
    hl_tokens_2.append(i)

In [118]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

# Init Lemmatizer 1
lemmatizer = WordNetLemmatizer()

hl_lemmatized_1 = []
for tokens in hl_tokens_1:
    lemm = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in tokens]
    hl_lemmatized_1.append(lemm)

 # Init Lemmatizer 2 
hl_lemmatized_2 = []
for tokens in hl_tokens_2:
    lemm = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in tokens]
    hl_lemmatized_2.append(lemm)

In [119]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
import numpy as np
max_features = 2000
max_token = len (hl_lemmatized_1)
tokenizer = Tokenizer (num_words= max_features)
tokenizer.fit_on_texts(hl_lemmatized_1)
sequences = tokenizer.texts_to_sequences(hl_lemmatized_1)
X_train = pad_sequences(sequences, maxlen = max_token)

####

max_features = 2000
max_token = len (hl_lemmatized_1)
tokenizer = Tokenizer (num_words= max_features)
tokenizer.fit_on_texts(hl_lemmatized_2)
sequences = tokenizer.texts_to_sequences(hl_lemmatized_2)
X_test = pad_sequences(sequences, maxlen = max_token)

In [3]:
Y_train = df_1['is_sarcastic'].values
Y_train = np.vstack(Y_train)
#X_trai,X_tes,Y_trai,Y_tes = train_test_split(X_train,Y_train,test_size=0.3, random_state = 42)
print (X_train)

#######
Y_test = df_2['is_sarcastic'].values
Y_test = np.vstack(Y_test)

NameError: ignored

In [121]:
len(X_test)

2159

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf  = RandomForestClassifier(n_estimators=300)
clf.fit(X_train, Y_train)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [1]:
from sklearn.preprocessing import LabelEncoder,MinMaxScaler

In [2]:
le = LabelEncoder()
y_result=le.fit_transform(Y_test)
y_pred=clf.predict(X_test)

NameError: ignored

In [ ]:
print("Result: ", (y_pred == y_result).tolist().count(True)/len(y_result))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_result,y_pred))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(ngram_range=(1,2), max_features=50000, min_df=2)
tfidf

In [ ]:
clf = LogisticRegression(C=1, n_jobs=4, solver='lbfgs', random_state=17, verbose=1)
tfidf_logit_pipeline = Pipeline([('tf_idf', tfidf),
                                 ('logistic', clf)])
tfidf_logit_pipeline.fit(X_train,Y_train)

In [ ]:
pred = tfidf_logit_pipeline.predict(test_X)
pred

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, pred)